importing packages

In [1]:
import numpy as np
import string 
import scipy.io
import os
import re

import nilearn.signal

/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Reading brain Data:

In [535]:
subject_id = "8"
datadir = "../data/"

fMRI_preprocessing_mode = "mean"#"mean" 
fMRI_preprocessing_flags= {"none":{'detrend':False, 'standard':False,'highpass':None},
                     "detrend":{'detrend':True, 'standard':False,'highpass':None},
                     "detrend_filter":{'detrend':True, 'standard':False,'highpass':0.005},
                     "detrend_filter_std":{'detrend':True, 'standard':True,'highpass':0.005}}

subject_file = scipy.io.loadmat(datadir+'subject_'+subject_id+'.mat') 

sub_id_index = 0
#nTRs
number_of_scans_index = 1
#nvoxels
number_of_voxels = 2
x_dim_index = 3
y_dim_index = 4
z_dim_index = 5
#colToCoord
colToCoord_index = 6
#coordToCol
coordToCol_index = 7
ROInumToName_index = 8
ROInumsToName_3d_index = 9
ROINames_index = 10
voxel_size_index = 11
#the map to the MNI space
matrix_index = 12

activations = subject_file['data']
subject_id = subject_file['meta'][0][0][sub_id_index][0][0]
number_of_scans = subject_file['meta'][0][0][number_of_scans_index][0][0]
number_of_voxels = subject_file['meta'][0][0][number_of_voxels][0][0]
x_dim = subject_file['meta'][0][0][x_dim_index][0][0]
y_dim = subject_file['meta'][0][0][y_dim_index][0][0]
z_dim = subject_file['meta'][0][0][z_dim_index][0][0]
coords = subject_file['meta'][0][0][colToCoord_index]
named_areas = subject_file['meta'][0][0][ROInumToName_index]
column_map = subject_file['meta'][0][0][coordToCol_index].shape
area_names_list = subject_file['meta'][0][0][ROINames_index][0]
voxel_size = subject_file['meta'][0][0][voxel_size_index]
nmi_matrix = subject_file['meta'][0][0][matrix_index]

Reading Story Feature:

In [536]:
index = 0
word_index = 0
time_index = 1
subject_file['words'][0][index][word_index][0][0][0]
story_features = scipy.io.loadmat('../data/story_features.mat') 
part_of_speaches_feature_id = 8
speach_feature_id = 1
part_of_speaches = story_features['features'][0][part_of_speaches_feature_id][1][0]
part_of_speaches_features = story_features['features'][0][part_of_speaches_feature_id][2]
speaches = story_features['features'][0][speach_feature_id][1][0]
speach_features = story_features['features'][0][speach_feature_id][2]

In [537]:
speaches

array([array(['speak_sticky'], dtype='<U12'),
       array(['speak'], dtype='<U5')], dtype=object)

In [538]:
print("How many words are there in the story? aligned with the brain data?")
subject_file['words'].shape

How many words are there in the story? aligned with the brain data?


(1, 5176)

In [539]:
actual_words = []
word_times = []
word_pos = []
speach_states = []
for i in np.arange(subject_file['words'].shape[1]):
    actual_words.append(subject_file['words'][0][i][word_index][0][0][0])
    word_times.append(subject_file['words'][0][i][time_index][0][0])
    word_pos.append(part_of_speaches_features[i])
    speach_states.append(speach_features[i])

Group time steps into blocks according to the fMRI experiment:

In [540]:
blocks = subject_file['time'][:,1]
scan_times = subject_file['time'][:,0]

block_ends = {}
block_ends_indexes = {}


for block_id in np.arange(1,5):
    block_ends_indexes[block_id] = np.where(scan_times == np.max(scan_times[np.where(blocks == block_id )]))[0]
    block_ends[block_id] = scan_times[block_ends_indexes[block_id]]+2


print(block_ends)
print(block_ends_indexes)

{1: array([680], dtype=uint16), 2: array([1384], dtype=uint16), 3: array([1942], dtype=uint16), 4: array([2702], dtype=uint16)}
{1: array([339]), 2: array([691]), 3: array([970]), 4: array([1350])}


group words and story features into blocks:

In [541]:
block_texts = {1:[],2:[],3:[],4:[]}
block_pos = {1:[],2:[],3:[],4:[]}
block_speach_state = {1:[],2:[],3:[],4:[]}
block_steps = {1:[],2:[],3:[],4:[]}
block_id = 1
for index in np.arange(len(actual_words)):
    if word_times[index] > block_ends[block_id]:
        block_id += 1
    block_texts[block_id].append(str(actual_words[index].encode("ascii",'ignore').decode()))
    block_pos[block_id].append(word_pos[index])
    block_speach_state[block_id].append(speach_states[index])
    block_steps[block_id].append(index)

In [542]:
len(list(zip(block_steps[1],block_texts[1])))

1303

Prepare input stimuli for the computational model (character input language model):

In [543]:
block_sentences = {1:[],2:[],3:[],4:[]}
translated_block_texts = {1:[],2:[],3:[],4:[]}
translated_block_pos = {1:[],2:[],3:[],4:[]}
translated_block_texts_steps = {1:[],2:[],3:[],4:[]}
translator = str.maketrans('', '', string.punctuation)


for block_id in np.arange(1,5):
    new_sentence = []
    for word,step,pos in zip(block_texts[block_id],block_steps[block_id],block_pos[block_id]):
        splitted_word = re.split("([A-Za-z\']+)", word.strip())
        tranlated_word = word.translate(translator)
        print("step:",step, "splited word:",splitted_word)
        last_w = ""
        for w in splitted_word:
            if len(w) > 0:
                last_w = w
                new_sentence.append(w)
                translated_block_texts[block_id].append(w)
                translated_block_texts_steps[block_id].append(step)
                translated_block_pos[block_id].append(pos)
        if last_w in ".?!":
            block_sentences[block_id].append(new_sentence)
            new_sentence = []
        
        """
        if len(word) == 1:
            tranlated_word = word
        if len(tranlated_word.strip()) > 0:
            translated_block_texts[block_id].append(tranlated_word)
            translated_block_texts_steps[block_id].append(step)
            translated_block_pos[block_id].append(pos)
            
        if len(tranlated_word.strip()) > 0:
            new_sentence.append(tranlated_word)
        if word.endswith("."):
            new_sentence.append(".")
            translated_block_texts[block_id].append(".")
            translated_block_texts_steps[block_id].append(step)
            translated_block_pos[block_id].append(np.zeros_like(pos))

            block_sentences[block_id].append(new_sentence)
            new_sentence = []
        """
    if len(new_sentence ) > 0:
            block_sentences[block_id].append(new_sentence)

    
    

step: 0 splited word: ['', 'Harry', '']
step: 1 splited word: ['', 'had', '']
step: 2 splited word: ['', 'never', '']
step: 3 splited word: ['', 'believed', '']
step: 4 splited word: ['', 'he', '']
step: 5 splited word: ['', 'would', '']
step: 6 splited word: ['', 'meet', '']
step: 7 splited word: ['', 'a', '']
step: 8 splited word: ['', 'boy', '']
step: 9 splited word: ['', 'he', '']
step: 10 splited word: ['', 'hated', '']
step: 11 splited word: ['', 'more', '']
step: 12 splited word: ['', 'than', '']
step: 13 splited word: ['', 'Dudley', ',']
step: 14 splited word: ['', 'but', '']
step: 15 splited word: ['', 'that', '']
step: 16 splited word: ['', 'was', '']
step: 17 splited word: ['', 'before', '']
step: 18 splited word: ['', 'he', '']
step: 19 splited word: ['', 'met', '']
step: 20 splited word: ['', 'Draco', '']
step: 21 splited word: ['', 'Malfoy', '.']
step: 22 splited word: ['', 'Still', ',']
step: 23 splited word: ['', 'first', '-', 'year', '']
step: 24 splited word: ['', 'Gr

step: 351 splited word: ['', 'This', '']
step: 352 splited word: ['', 'was', '']
step: 353 splited word: ['', 'something', '']
step: 354 splited word: ['', 'you', '']
step: 355 splited word: ['', "couldn't", '']
step: 356 splited word: ['', 'learn', '']
step: 357 splited word: ['', 'by', '']
step: 358 splited word: ['', 'heart', '']
step: 359 splited word: ['', 'out', '']
step: 360 splited word: ['', 'of', '']
step: 361 splited word: ['', 'a', '']
step: 362 splited word: ['', 'book', '']
step: 363 splited word: ['--']
step: 364 splited word: ['', 'not', '']
step: 365 splited word: ['', 'that', '']
step: 366 splited word: ['', 'she', '']
step: 367 splited word: ['', "hadn't", '']
step: 368 splited word: ['', 'tried', '.']
step: 369 splited word: ['', 'At', '']
step: 370 splited word: ['', 'breakfast', '']
step: 371 splited word: ['', 'on', '']
step: 372 splited word: ['', 'Thursday', '']
step: 373 splited word: ['', 'she', '']
step: 374 splited word: ['', 'bored', '']
step: 375 splited 

step: 601 splited word: ['', 'Malfoy', ',']
step: 602 splited word: ['', 'but', '']
step: 603 splited word: ['', 'Professor', '']
step: 604 splited word: ['', 'McGonagall', ',']
step: 605 splited word: ['', 'who', '']
step: 606 splited word: ['', 'could', '']
step: 607 splited word: ['', 'spot', '']
step: 608 splited word: ['', 'trouble', '']
step: 609 splited word: ['', 'quicker', '']
step: 610 splited word: ['', 'than', '']
step: 611 splited word: ['', 'any', '']
step: 612 splited word: ['', 'teacher', '']
step: 613 splited word: ['', 'in', '']
step: 614 splited word: ['', 'the', '']
step: 615 splited word: ['', 'school', ',']
step: 616 splited word: ['', 'was', '']
step: 617 splited word: ['', 'there', '']
step: 618 splited word: ['', 'in', '']
step: 619 splited word: ['', 'a', '']
step: 620 splited word: ['', 'flash', '.']
step: 621 splited word: ['+']
step: 622 splited word: ['"', "What's", '']
step: 623 splited word: ['', 'going', '']
step: 624 splited word: ['', 'on', '?"']
step

step: 851 splited word: ['"', 'UP', '!"']
step: 852 splited word: ['', 'everyone', '']
step: 853 splited word: ['', 'shouted', '.']
step: 854 splited word: ['+']
step: 855 splited word: ['', "Harry's", '']
step: 856 splited word: ['', 'broom', '']
step: 857 splited word: ['', 'jumped', '']
step: 858 splited word: ['', 'into', '']
step: 859 splited word: ['', 'his', '']
step: 860 splited word: ['', 'hand', '']
step: 861 splited word: ['', 'at', '']
step: 862 splited word: ['', 'once', ',']
step: 863 splited word: ['', 'but', '']
step: 864 splited word: ['', 'it', '']
step: 865 splited word: ['', 'was', '']
step: 866 splited word: ['', 'one', '']
step: 867 splited word: ['', 'of', '']
step: 868 splited word: ['', 'the', '']
step: 869 splited word: ['', 'few', '']
step: 870 splited word: ['', 'that', '']
step: 871 splited word: ['', 'did', '.']
step: 872 splited word: ['', 'Hermione', '']
step: 873 splited word: ['', "Granger's", '']
step: 874 splited word: ['', 'had', '']
step: 875 split

step: 1225 splited word: ['', 'laughter', '.']
step: 1226 splited word: ['+']
step: 1227 splited word: ['"', 'Did', '']
step: 1228 splited word: ['', 'you', '']
step: 1229 splited word: ['', 'see', '']
step: 1230 splited word: ['', 'his', '']
step: 1231 splited word: ['', 'face', ',']
step: 1232 splited word: ['', 'the', '']
step: 1233 splited word: ['', 'great', '']
step: 1234 splited word: ['', 'lump', '?"']
step: 1235 splited word: ['+']
step: 1236 splited word: ['', 'The', '']
step: 1237 splited word: ['', 'other', '']
step: 1238 splited word: ['', 'Slytherins', '']
step: 1239 splited word: ['', 'joined', '']
step: 1240 splited word: ['', 'in', '.']
step: 1241 splited word: ['+']
step: 1242 splited word: ['"', 'Shut', '']
step: 1243 splited word: ['', 'up', ',']
step: 1244 splited word: ['', 'Malfoy', ',"']
step: 1245 splited word: ['', 'snapped', '']
step: 1246 splited word: ['', 'Parvati', '']
step: 1247 splited word: ['', 'Patil', '.']
step: 1248 splited word: ['+']
step: 1249 s

...

step: 3350 splited word: ['@', 'only', '']
step: 3351 splited word: ['', 'care', '']
step: 3352 splited word: ['', 'about', '']
step: 3353 splited word: ['', 'yourselves', ',']
step: 3354 splited word: ['', 'I', '']
step: 3355 splited word: ['', "don't", '']
step: 3356 splited word: ['', 'want', '']
step: 3357 splited word: ['', 'Slytherin', '']
step: 3358 splited word: ['', 'to', '']
step: 3359 splited word: ['', 'win', '']
step: 3360 splited word: ['', 'the', '']
step: 3361 splited word: ['', 'House', '']
step: 3362 splited word: ['', 'Cup', ',']
step: 3363 splited word: ['', 'and', '']
step: 3364 splited word: ['', "you'll", '']
step: 3365 splited word: ['', 'lose', '']
step: 3366 splited word: ['', 'all', '']
step: 3367 splited word: ['', 'the', '']
step: 3368 splited word: ['', 'points', '']
step: 3369 splited word: ['', 'I', '']
step: 3370 splited word: ['', 'got', '']
step: 3371 splited word: ['', 'from', '']
step: 3372 splited word: ['', 'Professor', '']
step: 3373 splited word

step: 3725 splited word: ['', 'Neville', '.']
step: 3726 splited word: ['+']
step: 3727 splited word: ['"', 'If', '']
step: 3728 splited word: ['', 'either', '']
step: 3729 splited word: ['', 'of', '']
step: 3730 splited word: ['', 'you', '']
step: 3731 splited word: ['', 'get', '']
step: 3732 splited word: ['', 'us', '']
step: 3733 splited word: ['', 'caught', ',']
step: 3734 splited word: ['', "I'll", '']
step: 3735 splited word: ['', 'never', '']
step: 3736 splited word: ['', 'rest', '']
step: 3737 splited word: ['', 'until', '']
step: 3738 splited word: ['', "I've", '']
step: 3739 splited word: ['', 'learned', '']
step: 3740 splited word: ['', 'that', '']
step: 3741 splited word: ['', 'Curse', '']
step: 3742 splited word: ['', 'of', '']
step: 3743 splited word: ['', 'the', '']
step: 3744 splited word: ['', 'Bogies', '']
step: 3745 splited word: ['', 'Quirrell', '']
step: 3746 splited word: ['', 'told', '']
step: 3747 splited word: ['', 'us', '']
step: 3748 splited word: ['', 'about

step: 4099 splited word: ['--']
step: 4100 splited word: ['', 'they', '']
step: 4101 splited word: ['', 'swung', '']
step: 4102 splited word: ['', 'around', '']
step: 4103 splited word: ['', 'the', '']
step: 4104 splited word: ['', 'doorpost', '']
step: 4105 splited word: ['', 'and', '']
step: 4106 splited word: ['', 'galloped', '']
step: 4107 splited word: ['', 'down', '']
step: 4108 splited word: ['', 'one', '']
step: 4109 splited word: ['', 'corridor', '']
step: 4110 splited word: ['', 'then', '']
step: 4111 splited word: ['', 'another', ',']
step: 4112 splited word: ['', 'Harry', '']
step: 4113 splited word: ['', 'in', '']
step: 4114 splited word: ['', 'the', '']
step: 4115 splited word: ['', 'lead', ',']
step: 4116 splited word: ['', 'without', '']
step: 4117 splited word: ['', 'any', '']
step: 4118 splited word: ['', 'idea', '']
step: 4119 splited word: ['', 'where', '']
step: 4120 splited word: ['', 'they', '']
step: 4121 splited word: ['', 'were', '']
step: 4122 splited word: [

step: 4474 splited word: ['', 'as', '']
step: 4475 splited word: ['', 'he', '']
step: 4476 splited word: ['', 'could', '']
step: 4477 splited word: ['', 'toward', '']
step: 4478 splited word: ['', "Peeves's", '']
step: 4479 splited word: ['', 'shouts', '.']
step: 4480 splited word: ['+']
step: 4481 splited word: ['"', 'Oh', ',']
step: 4482 splited word: ['', 'move', '']
step: 4483 splited word: ['', 'over', ',"']
step: 4484 splited word: ['', 'Hermione', '']
step: 4485 splited word: ['', 'snarled', '.']
step: 4486 splited word: ['', 'She', '']
step: 4487 splited word: ['', 'grabbed', '']
step: 4488 splited word: ['', "Harry's", '']
step: 4489 splited word: ['', 'wand', ',']
step: 4490 splited word: ['', 'tapped', '']
step: 4491 splited word: ['', 'the', '']
step: 4492 splited word: ['', 'lock', ',']
step: 4493 splited word: ['', 'and', '']
step: 4494 splited word: ['', 'whispered', ',']
step: 4495 splited word: ['@"', 'Alohomora', '!"']
step: 4496 splited word: ['+']
step: 4497 splited

step: 4795 splited word: ['', 'growls', '']
step: 4796 splited word: ['', 'meant', '.']
step: 4797 splited word: ['+']
step: 4798 splited word: ['', 'Harry', '']
step: 4799 splited word: ['', 'groped', '']
step: 4800 splited word: ['', 'for', '']
step: 4801 splited word: ['', 'the', '']
step: 4802 splited word: ['', 'doorknob', '']
step: 4803 splited word: ['--']
step: 4804 splited word: ['', 'between', '']
step: 4805 splited word: ['', 'Filch', '']
step: 4806 splited word: ['', 'and', '']
step: 4807 splited word: ['', 'death', ',']
step: 4808 splited word: ['', "he'd", '']
step: 4809 splited word: ['', 'take', '']
step: 4810 splited word: ['', 'Filch', '.']
step: 4811 splited word: ['+']
step: 4812 splited word: ['', 'They', '']
step: 4813 splited word: ['', 'fell', '']
step: 4814 splited word: ['', 'backward', '']
step: 4815 splited word: ['--']
step: 4816 splited word: ['', 'Harry', '']
step: 4817 splited word: ['', 'slammed', '']
step: 4818 splited word: ['', 'the', '']
step: 4819 

In [544]:
block_sentences[1] 

[['Harry',
  'had',
  'never',
  'believed',
  'he',
  'would',
  'meet',
  'a',
  'boy',
  'he',
  'hated',
  'more',
  'than',
  'Dudley',
  ',',
  'but',
  'that',
  'was',
  'before',
  'he',
  'met',
  'Draco',
  'Malfoy',
  '.'],
 ['Still',
  ',',
  'first',
  '-',
  'year',
  'Gryffindors',
  'only',
  'had',
  'Potions',
  'with',
  'the',
  'Slytherins',
  ',',
  'so',
  'they',
  "didn't",
  'have',
  'to',
  'put',
  'up',
  'with',
  'Malfoy',
  'much',
  '.'],
 ['Or',
  'at',
  'least',
  ',',
  'they',
  "didn't",
  'until',
  'they',
  'spotted',
  'a',
  'notice',
  'pinned',
  'up',
  'in',
  'the',
  'Gryffindor',
  'common',
  'room',
  'that',
  'made',
  'them',
  'all',
  'groan',
  '.'],
 ['Flying',
  'lessons',
  'would',
  'be',
  'starting',
  'on',
  'Thursday',
  '--',
  'and',
  'Gryffindor',
  'and',
  'Slytherin',
  'would',
  'be',
  'learning',
  'together',
  '.'],
 ['+', '"', 'Typical', ',"', 'said', 'Harry', 'darkly', '.'],
 ['"', 'Just', 'what', 'I'

In [545]:
block_text = {}
for block_id in np.arange(1,5):
    block_text[block_id] = ' '.join(translated_block_texts[block_id])
    
print(block_text)

{1: 'Harry had never believed he would meet a boy he hated more than Dudley , but that was before he met Draco Malfoy . Still , first - year Gryffindors only had Potions with the Slytherins , so they didn\'t have to put up with Malfoy much . Or at least , they didn\'t until they spotted a notice pinned up in the Gryffindor common room that made them all groan . Flying lessons would be starting on Thursday -- and Gryffindor and Slytherin would be learning together . + " Typical ," said Harry darkly . " Just what I always wanted . To make a fool of myself on a broomstick in front of Malfoy ." + He had been looking forward to learning to fly more than anything else . " You don\'t know that you\'ll make a fool of yourself ," said Ron reasonably . " Anyway , I know Malfoy\'s always going on about how good he is at Quidditch , but I bet that\'s all talk ." + Malfoy certainly did talk about flying a lot . He complained loudly about first years never getting on the House Quidditch teams and to

In [546]:
processed_data_dir = "../processed_data/subject_"+str(subject_id)
try:
    os.stat(processed_data_dir)
except:
    os.makedirs(processed_data_dir) 
for block_id in np.arange(1,5):
    
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_sentences",block_sentences[block_id])
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_text",translated_block_texts[block_id])
    np.save(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_pos",translated_block_pos[block_id])

    with open(processed_data_dir+"/subject_"+str(subject_id)+"_block_"+str(block_id)+"_rawtext.txt","w") as fout:
        fout.write(block_text[block_id])

In [547]:
print(np.asarray(translated_block_texts[1])[np.where(np.asarray(translated_block_texts_steps[1]) == 117)])
print(np.asarray(translated_block_texts_steps[1])[np.where(np.asarray(translated_block_texts_steps[1]) == 117)])

["don't"]
[117]


In [548]:
translated_block_texts

{1: ['Harry',
  'had',
  'never',
  'believed',
  'he',
  'would',
  'meet',
  'a',
  'boy',
  'he',
  'hated',
  'more',
  'than',
  'Dudley',
  ',',
  'but',
  'that',
  'was',
  'before',
  'he',
  'met',
  'Draco',
  'Malfoy',
  '.',
  'Still',
  ',',
  'first',
  '-',
  'year',
  'Gryffindors',
  'only',
  'had',
  'Potions',
  'with',
  'the',
  'Slytherins',
  ',',
  'so',
  'they',
  "didn't",
  'have',
  'to',
  'put',
  'up',
  'with',
  'Malfoy',
  'much',
  '.',
  'Or',
  'at',
  'least',
  ',',
  'they',
  "didn't",
  'until',
  'they',
  'spotted',
  'a',
  'notice',
  'pinned',
  'up',
  'in',
  'the',
  'Gryffindor',
  'common',
  'room',
  'that',
  'made',
  'them',
  'all',
  'groan',
  '.',
  'Flying',
  'lessons',
  'would',
  'be',
  'starting',
  'on',
  'Thursday',
  '--',
  'and',
  'Gryffindor',
  'and',
  'Slytherin',
  'would',
  'be',
  'learning',
  'together',
  '.',
  '+',
  '"',
  'Typical',
  ',"',
  'said',
  'Harry',
  'darkly',
  '.',
  '"',
  'Just

In [549]:
translated_block_texts_steps[1]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 21,
 22,
 22,
 23,
 23,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 40,
 41,
 42,
 43,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 78,
 79,
 80,
 80,
 80,
 81,
 82,
 83,
 83,
 84,
 84,
 85,
 86,
 87,
 88,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 115,
 116,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 125,
 126,
 127,
 128,
 128,
 129,
 129,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 157,
 158,
 159,
 160,
 161,
 16

In [550]:
class Scan(object):
    def __init__(self,activations,timestamp, step,prev_words=None,next_words=None,all_words=None,all_pos=None,all_speak_features=None,current_translated_words=None):
        self.activations = activations
        self.timestamp = timestamp
        self.prev_words = prev_words
        self.next_words = next_words
        self.step = step
        self.all_words = all_words
        self.current_translated_words = current_translated_words
        self.all_pos = all_pos
        self.all_speak_features = all_speak_features
        self.brain3d = None
        


In [551]:
vocab = []
translated_words = []
with open(processed_data_dir+"/subject_"+subject_id+"_harry_potter_vocab.txt","w") as fout:
    for word in actual_words:
        w = word.encode("ascii",'ignore').decode()
        if len(w) > 1:
            w = w #.translate(translator)
        if len(w) > 0:
            fout.write(w+'\n')
            print(w)
            vocab.append(w)

        translated_words.append(w)
        

Harry
had
never
believed
he
would
meet
a
boy
he
hated
more
than
Dudley,
but
that
was
before
he
met
Draco
Malfoy.
Still,
first-year
Gryffindors
only
had
Potions
with
the
Slytherins,
so
they
didn't
have
to
put
up
with
Malfoy
much.
Or
at
least,
they
didn't
until
they
spotted
a
notice
pinned
up
in
the
Gryffindor
common
room
that
made
them
all
groan.
Flying
lessons
would
be
starting
on
Thursday
--
and
Gryffindor
and
Slytherin
would
be
learning
together.
+
"Typical,"
said
Harry
darkly.
"Just
what
I
always
wanted.
To
make
a
fool
of
myself
on
a
broomstick
in
front
of
Malfoy."
+
He
had
been
looking
forward
to
learning
to
fly
more
than
anything
else.
"You
don't
know
that
you'll
make
a
fool
of
yourself,"
said
Ron
reasonably.
"Anyway,
I
know
Malfoy's
always
going
on
about
how
good
he
is
at
Quidditch,
but
I
bet
that's
all
talk."
+
Malfoy
certainly
did
talk
about
flying
a
lot.
He
complained
loudly
about
first
years
never
getting
on
the
House
Quidditch
teams
and
told
long,
boastful
stories
that
alway

motion,
the
ball
rise
up
in
the
air
and
then
start
to
fall.
He
leaned
forward
and
pointed
his
broom
handle
down
--
next
second
he
was
gathering
speed
in
a
steep
dive,
racing
the
ball
--
wind
whistled
in
his
ears,
mingled
with
the
screams
of
people
watching
--
he
stretched
out
his
hand
--
a
foot
from
the
ground
he
caught
it,
just
in
time
to
pull
his
broom
straight,
and
he
toppled
gently
onto
the
grass
with
the
Remembrall
clutched
safely
in
his
fist.
+
"HARRY
POTTER!"
+
His
heart
sank
faster
than
he'd
just
dived.
Professor
McGonagall
was
running
toward
them.
He
got
to
his
feet,
trembling.
+
@"Never
--
in
all
my
time
at
Hogwarts
--"
Professor
McGonagall
was
almost
speechless
with
shock,
and
her
glasses
flashed
furiously,
"--
how
@dare
you
--
might
have
broken
your
neck
--"
+
"It
wasn't
his
fault,
Professor
--"
+
"Be
quiet,
Miss
Patil
--"
+
"But
Malfoy
--"
"That's
enough,
Mr.
Weasley.
Potter,
follow
me,
now."
+
Harry
caught
sight
of
Malfoy,
Crabbe,
and
Goyle's
triumphant
faces
as
he
left,


The
password's
'Pig
snout'
but
it
won't
help
you
now,
the
Fat
Lady's
gone
off
somewhere."
+
"How's
your
arm?"
said
Harry.
+
"Fine,"
said
Neville,
showing
them.
"Madam
Pomfrey
mended
it
in
about
a
minute."
+
"Good
--
well,
look,
Neville,
we've
got
to
be
somewhere,
we'll
see
you
later
--"
+
"Don't
leave
me!"
said
Neville,
scrambling
to
his
feet,
"I
don't
want
to
stay
here
alone,
the
Bloody
Baron's
been
past
twice
already."
+
Ron
looked
at
his
watch
and
then
glared
furiously
at
Hermione
and
Neville.
+
"If
either
of
you
get
us
caught,
I'll
never
rest
until
I've
learned
that
Curse
of
the
Bogies
Quirrell
told
us
about,
and
used
it
on
you."
+
Hermione
opened
her
mouth,
perhaps
to
tell
Ron
exactly
how
to
use
the
Curse
of
the
Bogies,
but
Harry
hissed
at
her
to
be
quiet
and
beckoned
them
all
forward.
+
They
flitted
along
corridors
striped
with
bars
of
moonlight
from
the
high
windows.
At
every
turn
Harry
expected
to
run
into
Filch
or
Mrs.
Norris,
but
they
were
lucky.
They
sped
up
a
staircase
to
t

In [552]:
len(word_pos)
len(translated_words)
processed_data_dir

'../processed_data/subject_8'

Detrending brain activations:

In [553]:
block_activations = {1:[],2:[],3:[],4:[]}
block_id = 1
for j in np.arange(0,len(activations)):
    if j > block_ends_indexes[block_id][0]:
        block_id +=1
    block_activations[block_id].append(activations[j])

print(len(block_activations[1]))
print(len(block_activations[2]))
print(len(block_activations[3]))
print(len(block_activations[4]))

clean_block_activations = {1:[],2:[],3:[],4:[]}
all_clean_activations = []
for block_id in block_activations.keys():
    print("block id:",block_id)

    clean_activations = list(block_activations[block_id])
    if fMRI_preprocessing_mode == "none":
        clean_activations = np.asarray(clean_activations)
        
        indexes = np.where((np.min(clean_activations,axis=0) < np.max(clean_activations,axis=0)) == True)[0]
        indexes_p = np.where((np.min(clean_activations,axis=0) == np.max(clean_activations,axis=0)) == True)[0]

        clean_activations[:,indexes_p] = 0.0
        clean_activations[:,indexes] = \
        (clean_activations[:,indexes] - np.min(clean_activations[:,indexes],axis=0)) / \
        (np.max(clean_activations[:,indexes],axis=0) - np.min(clean_activations[:,indexes],axis=0))
    elif fMRI_preprocessing_mode == "mean":
        clean_activations = clean_activations - np.mean(clean_activations,axis=0)
    else:
        clean_activations = nilearn.signal.clean(block_activations[block_id], sessions=None, 
                             detrend=fMRI_preprocessing_flags[fMRI_preprocessing_mode]['detrend'], 
                             standardize=fMRI_preprocessing_flags[fMRI_preprocessing_mode]['standard'], 
                             confounds=None, low_pass=None, 
                             high_pass=fMRI_preprocessing_flags[fMRI_preprocessing_mode]['highpass'], 
                            t_r=2.0, ensure_finite=False)
    clean_block_activations[block_id] = clean_activations
    all_clean_activations.extend(clean_activations)

all_clean_activations = np.asarray(all_clean_activations)
print(len(all_clean_activations))

print("min:",np.min(all_clean_activations,axis=1))
print("max:",np.max(all_clean_activations,axis=1))

340
352
279
380
block id: 1
block id: 2
block id: 3
block id: 4
1351
min: [-72.74411765 -74.         -93.32058824 ... -89.68421053 -46.76578947
 -67.42631579]
max: [180.75        89.81470588 110.03235294 ... 124.37631579 148.06315789
  69.65789474]


Saving Scan Objects:

In [554]:
block_scans = {1:[],2:[],3:[],4:[]}
block_id = 1

j = np.where(scan_times == word_times[0])[0]
for i in np.arange(0,len(actual_words)):
    if word_times[i] > scan_times[j]:
        j = np.where(scan_times == word_times[i])[0]
    if j > block_ends_indexes[block_id][0]:
            block_id +=1
    #print(block_id,word_times[i], scan_times[j][0])
    #print(word_times[i], scan_times[j])
    if word_times[i] == scan_times[j][0]:
        #print(i,j)
        
        k = i
        current_step_words = []
        current_step_pos = []
        current_step_speach_states = []
        translated_current_words = []
        counter = 0
        while k < len(actual_words) and counter < 4 :
            current_step_words.append(actual_words[k])
            current_step_pos.append(word_pos[k])
            current_step_speach_states.append(speach_states[k])
            
            translated_text_index = np.where(np.asarray(translated_block_texts_steps[block_id]) == k)
            if len(translated_text_index) > 0:
                print(block_id,translated_text_index)
                translated_current_words.append(np.asarray(translated_block_texts[block_id])[translated_text_index])
            
            k += 1
            counter +=1
            
            
        k = i+4
        next_step_words = []
        next_step_pos = []
        next_step_speach_states = []
        counter = 0
        while k >= 0 and k < len(actual_words) and counter < 4 :
            next_step_words.append(actual_words[k])
            next_step_pos.append(word_pos[k])
            next_step_speach_states.append(speach_states[k])
            k += 1
            counter +=1
         
        k = i-4
        prev_step_words = []
        prev_step_pos = []
        prev_step_speach_states = []
        counter = 0
        while k >= 0 and k < len(actual_words) and counter < 4 :
            prev_step_words.append(actual_words[k])
            prev_step_pos.append(word_pos[k])
            prev_step_speach_states.append(speach_states[k])
            k += 1
            counter +=1
            
            
        #print(translated_current_words)
        scan = Scan(all_clean_activations[j],scan_times[j][0],i,
                        prev_step_words,
                        next_step_words,
                        current_step_words,
                        current_step_pos,
                        current_step_speach_states,
                        translated_current_words) 
                   
        
        # print(i,j,block_ends_indexes[block_id][0])
        
        j += 1
        block_scans[block_id].append(scan)
        


1 (array([0]),)
1 (array([1]),)
1 (array([2]),)
1 (array([3]),)
1 (array([4]),)
1 (array([5]),)
1 (array([6]),)
1 (array([7]),)
1 (array([8]),)
1 (array([9]),)
1 (array([10]),)
1 (array([11]),)
1 (array([12]),)
1 (array([13, 14]),)
1 (array([15]),)
1 (array([16]),)
1 (array([17]),)
1 (array([18]),)
1 (array([19]),)
1 (array([20]),)
1 (array([21]),)
1 (array([22, 23]),)
1 (array([24, 25]),)
1 (array([26, 27, 28]),)
1 (array([29]),)
1 (array([30]),)
1 (array([31]),)
1 (array([32]),)
1 (array([33]),)
1 (array([34]),)
1 (array([35, 36]),)
1 (array([37]),)
1 (array([38]),)
1 (array([39]),)
1 (array([40]),)
1 (array([41]),)
1 (array([42]),)
1 (array([43]),)
1 (array([44]),)
1 (array([45]),)
1 (array([46, 47]),)
1 (array([48]),)
1 (array([49]),)
1 (array([50, 51]),)
1 (array([52]),)
1 (array([53]),)
1 (array([54]),)
1 (array([55]),)
1 (array([56]),)
1 (array([57]),)
1 (array([58]),)
1 (array([59]),)
1 (array([60]),)
1 (array([61]),)
1 (array([62]),)
1 (array([63]),)
1 (array([64]),)
1 (array(

1 (array([526]),)
1 (array([527]),)
1 (array([528, 529]),)
1 (array([530]),)
1 (array([531]),)
1 (array([532]),)
1 (array([533]),)
1 (array([534]),)
1 (array([535]),)
1 (array([536]),)
1 (array([537]),)
1 (array([538]),)
1 (array([539]),)
1 (array([540]),)
1 (array([541, 542]),)
1 (array([543]),)
1 (array([544]),)
1 (array([545]),)
1 (array([546]),)
1 (array([547]),)
1 (array([548]),)
1 (array([549]),)
1 (array([550]),)
1 (array([551]),)
1 (array([552]),)
1 (array([553]),)
1 (array([554]),)
1 (array([555]),)
1 (array([556]),)
1 (array([557]),)
1 (array([558, 559]),)
1 (array([560]),)
1 (array([561]),)
1 (array([562]),)
1 (array([563]),)
1 (array([564]),)
1 (array([565]),)
1 (array([566]),)
1 (array([567, 568]),)
1 (array([569]),)
1 (array([570, 571]),)
1 (array([572]),)
1 (array([573, 574]),)
1 (array([575]),)
1 (array([576, 577]),)
1 (array([578, 579]),)
1 (array([580]),)
1 (array([581]),)
1 (array([582]),)
1 (array([583]),)
1 (array([584]),)
1 (array([585]),)
1 (array([586]),)
1 (arr

1 (array([1089]),)
1 (array([1090]),)
1 (array([1091]),)
1 (array([1092]),)
1 (array([1093]),)
1 (array([1094]),)
1 (array([1095]),)
1 (array([1096]),)
1 (array([1097]),)
1 (array([1098]),)
1 (array([1099]),)
1 (array([1100]),)
1 (array([1101]),)
1 (array([1102]),)
1 (array([1103]),)
1 (array([1104, 1105]),)
1 (array([1106]),)
1 (array([1107, 1108, 1109]),)
1 (array([1110]),)
1 (array([1111]),)
1 (array([1112]),)
1 (array([1113]),)
1 (array([1114, 1115]),)
1 (array([1116]),)
1 (array([1117]),)
1 (array([1118]),)
1 (array([1119]),)
1 (array([1120]),)
1 (array([1121, 1122]),)
1 (array([1123, 1124]),)
1 (array([1125]),)
1 (array([1126]),)
1 (array([1127, 1128]),)
1 (array([1129, 1130]),)
1 (array([1131]),)
1 (array([1132]),)
1 (array([1133, 1134]),)
1 (array([1135]),)
1 (array([1136]),)
1 (array([1137]),)
1 (array([1138, 1139]),)
1 (array([1140]),)
1 (array([1141]),)
1 (array([1142]),)
1 (array([1143]),)
1 (array([1144]),)
1 (array([1145]),)
1 (array([1146]),)
1 (array([1147]),)
1 (array(

2 (array([80]),)
2 (array([81]),)
2 (array([82]),)
2 (array([83, 84]),)
2 (array([85, 86]),)
2 (array([87]),)
2 (array([88]),)
2 (array([89, 90]),)
2 (array([91, 92]),)
2 (array([93]),)
2 (array([94]),)
2 (array([95]),)
2 (array([96]),)
2 (array([97, 98]),)
2 (array([99]),)
2 (array([100, 101, 102]),)
2 (array([103]),)
2 (array([104]),)
2 (array([105, 106]),)
2 (array([107, 108]),)
2 (array([109]),)
2 (array([110]),)
2 (array([111]),)
2 (array([112]),)
2 (array([113]),)
2 (array([114]),)
2 (array([115]),)
2 (array([116]),)
2 (array([117]),)
2 (array([118]),)
2 (array([119]),)
2 (array([120]),)
2 (array([121, 122]),)
2 (array([123]),)
2 (array([124]),)
2 (array([125]),)
2 (array([126, 127]),)
2 (array([128]),)
2 (array([129]),)
2 (array([130]),)
2 (array([131]),)
2 (array([132]),)
2 (array([133, 134]),)
2 (array([135]),)
2 (array([136]),)
2 (array([137]),)
2 (array([138]),)
2 (array([139]),)
2 (array([140]),)
2 (array([141]),)
2 (array([142]),)
2 (array([143]),)
2 (array([144]),)
2 (arr

2 (array([600, 601]),)
2 (array([602]),)
2 (array([603]),)
2 (array([604]),)
2 (array([605]),)
2 (array([606]),)
2 (array([607, 608]),)
2 (array([609, 610]),)
2 (array([611]),)
2 (array([612]),)
2 (array([613]),)
2 (array([614]),)
2 (array([615]),)
2 (array([616]),)
2 (array([617, 618]),)
2 (array([619]),)
2 (array([620]),)
2 (array([621]),)
2 (array([622]),)
2 (array([623]),)
2 (array([624]),)
2 (array([625]),)
2 (array([626]),)
2 (array([627]),)
2 (array([628]),)
2 (array([629]),)
2 (array([630, 631]),)
2 (array([632]),)
2 (array([633]),)
2 (array([634]),)
2 (array([635]),)
2 (array([636]),)
2 (array([637, 638]),)
2 (array([639]),)
2 (array([640]),)
2 (array([641]),)
2 (array([642, 643]),)
2 (array([644]),)
2 (array([645]),)
2 (array([646]),)
2 (array([647]),)
2 (array([648]),)
2 (array([649]),)
2 (array([650]),)
2 (array([651, 652]),)
2 (array([653]),)
2 (array([654]),)
2 (array([655]),)
2 (array([656]),)
2 (array([657]),)
2 (array([658]),)
2 (array([659]),)
2 (array([660]),)
2 (arr

...

3 (array([567]),)
3 (array([568]),)
3 (array([569]),)
3 (array([570]),)
3 (array([571]),)
3 (array([572]),)
3 (array([573]),)
3 (array([574, 575]),)
3 (array([576, 577]),)
3 (array([578]),)
3 (array([579]),)
3 (array([580]),)
3 (array([581]),)
3 (array([582]),)
3 (array([583]),)
3 (array([584]),)
3 (array([585, 586]),)
3 (array([587]),)
3 (array([588]),)
3 (array([589]),)
3 (array([590]),)
3 (array([591, 592]),)
3 (array([593]),)
3 (array([594]),)
3 (array([595]),)
3 (array([596, 597]),)
3 (array([598]),)
3 (array([599]),)
3 (array([600]),)
3 (array([601]),)
3 (array([602]),)
3 (array([603]),)
3 (array([604]),)
3 (array([605]),)
3 (array([606]),)
3 (array([607]),)
3 (array([608]),)
3 (array([609]),)
3 (array([610]),)
3 (array([611]),)
3 (array([612]),)
3 (array([613]),)
3 (array([614]),)
3 (array([615]),)
3 (array([616]),)
3 (array([617, 618, 619, 620, 621, 622]),)
3 (array([623]),)
3 (array([624]),)
3 (array([625]),)
3 (array([626, 627]),)
3 (array([628]),)
3 (array([629, 630, 631, 63

3 (array([1105, 1106]),)
3 (array([1107, 1108]),)
3 (array([1109]),)
3 (array([1110, 1111]),)
3 (array([1112]),)
3 (array([1113]),)
3 (array([1114]),)
3 (array([1115]),)
3 (array([1116]),)
3 (array([1117]),)
3 (array([1118, 1119]),)
3 (array([1120]),)
3 (array([1121, 1122, 1123]),)
3 (array([1124, 1125]),)
3 (array([1126]),)
3 (array([1127, 1128]),)
3 (array([1129]),)
3 (array([1130]),)
3 (array([1131]),)
3 (array([1132, 1133]),)
3 (array([1134]),)
3 (array([1135]),)
3 (array([1136]),)
3 (array([1137, 1138]),)
3 (array([1139, 1140]),)
3 (array([1141]),)
3 (array([1142]),)
3 (array([1143, 1144]),)
3 (array([1145]),)
3 (array([1146]),)
3 (array([1147]),)
3 (array([1148]),)
3 (array([1149]),)
3 (array([1150]),)
3 (array([1151, 1152]),)
3 (array([1153]),)
3 (array([1154, 1155]),)
3 (array([1156]),)
3 (array([1157]),)
3 (array([1158]),)
3 (array([1159]),)
3 (array([1160]),)
3 (array([1161]),)
3 (array([1162]),)
3 (array([1163, 1164]),)
3 (array([1165]),)
3 (array([1166, 1167]),)
3 (array([1

4 (array([434]),)
4 (array([435]),)
4 (array([436, 437]),)
4 (array([438]),)
4 (array([439]),)
4 (array([440]),)
4 (array([441]),)
4 (array([442]),)
4 (array([443]),)
4 (array([444]),)
4 (array([445]),)
4 (array([446]),)
4 (array([447]),)
4 (array([448]),)
4 (array([449]),)
4 (array([450]),)
4 (array([451]),)
4 (array([452]),)
4 (array([453]),)
4 (array([454]),)
4 (array([455]),)
4 (array([456]),)
4 (array([457]),)
4 (array([458]),)
4 (array([459, 460]),)
4 (array([461]),)
4 (array([462]),)
4 (array([463]),)
4 (array([464, 465]),)
4 (array([466]),)
4 (array([467]),)
4 (array([468]),)
4 (array([469]),)
4 (array([470]),)
4 (array([471]),)
4 (array([472]),)
4 (array([473]),)
4 (array([474]),)
4 (array([475]),)
4 (array([476]),)
4 (array([477]),)
4 (array([478]),)
4 (array([479]),)
4 (array([480]),)
4 (array([481]),)
4 (array([482]),)
4 (array([483]),)
4 (array([484]),)
4 (array([485]),)
4 (array([486]),)
4 (array([487]),)
4 (array([488]),)
4 (array([489, 490]),)
4 (array([491]),)
4 (array

4 (array([1036]),)
4 (array([1037, 1038, 1039]),)
4 (array([1040]),)
4 (array([1041, 1042]),)
4 (array([1043]),)
4 (array([1044]),)
4 (array([1045]),)
4 (array([1046]),)
4 (array([1047]),)
4 (array([1048]),)
4 (array([1049]),)
4 (array([1050]),)
4 (array([1051]),)
4 (array([1052]),)
4 (array([1053, 1054]),)
4 (array([1055]),)
4 (array([1056, 1057]),)
4 (array([1058, 1059]),)
4 (array([1060]),)
4 (array([1061]),)
4 (array([1062]),)
4 (array([1063]),)
4 (array([1064]),)
4 (array([1065]),)
4 (array([1066]),)
4 (array([1067]),)
4 (array([1068]),)
4 (array([1069]),)
4 (array([1070]),)
4 (array([1071]),)
4 (array([1072]),)
4 (array([1073, 1074]),)
4 (array([1075]),)
4 (array([1076, 1077]),)
4 (array([1078]),)
4 (array([1079]),)
4 (array([1080]),)
4 (array([1081]),)
4 (array([1082, 1083]),)
4 (array([1084]),)
4 (array([1085, 1086]),)
4 (array([1087, 1088]),)
4 (array([1089]),)
4 (array([1090]),)
4 (array([1091]),)
4 (array([1092]),)
4 (array([1093]),)
4 (array([1094]),)
4 (array([1095, 1096, 

4 (array([1521, 1522]),)
4 (array([1523]),)
4 (array([1524]),)
4 (array([1525]),)
4 (array([1526, 1527]),)
4 (array([1528]),)
4 (array([1529]),)
4 (array([1530, 1531]),)
4 (array([1532]),)
4 (array([1533]),)
4 (array([1534]),)
4 (array([1535]),)
4 (array([1536]),)
4 (array([1537]),)
4 (array([1538]),)
4 (array([1539]),)
4 (array([1540]),)
4 (array([1541]),)
4 (array([1542]),)
4 (array([1543]),)
4 (array([1544, 1545]),)
4 (array([1546]),)
4 (array([1547, 1548]),)
4 (array([1549]),)
4 (array([1550]),)
4 (array([1551]),)
4 (array([1552, 1553]),)
4 (array([1554]),)
4 (array([1555]),)
4 (array([1556, 1557]),)
4 (array([1558]),)
4 (array([1559, 1560]),)
4 (array([1561]),)
4 (array([1562, 1563]),)
4 (array([1564, 1565]),)
4 (array([1566]),)
4 (array([1567]),)
4 (array([1568]),)
4 (array([1569]),)
4 (array([1570]),)
4 (array([1571]),)
4 (array([1572, 1573]),)
4 (array([1574]),)
4 (array([1575, 1576]),)
4 (array([1577, 1578]),)
4 (array([1579]),)
4 (array([1580, 1581]),)
4 (array([1582, 1583]),

In [555]:
translated_block_texts[2][0]

'"'

In [556]:
translated_block_texts_steps[1]
print(len(block_text[4].split()))
print(len(translated_block_texts[4]))

for i in np.arange(len(block_text[1].split())):
    if block_text[1].split()[i] != translated_block_texts[1][i]:
        print(block_text[1].split()[i],translated_block_texts[1][i])


1766
1766


In [557]:
block_scans[1][0].current_translated_words

[array(['Harry'], dtype='<U13'),
 array(['had'], dtype='<U13'),
 array(['never'], dtype='<U13'),
 array(['believed'], dtype='<U13')]

In [558]:
np.save(processed_data_dir+"/"+fMRI_preprocessing_mode+"subject_"+subject_id+"_scan_objects",block_scans)

In [534]:
block_id = 1
print(translated_block_texts_steps[block_id])
np.where(np.asarray(translated_block_texts_steps[block_id]) == 1)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 13, 14, 15, 16, 17, 18, 19, 20, 21, 21, 22, 22, 23, 23, 23, 24, 25, 26, 27, 28, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 40, 41, 42, 43, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 78, 79, 80, 80, 80, 81, 82, 83, 83, 84, 84, 85, 86, 87, 88, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 115, 116, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 125, 126, 127, 128, 128, 129, 129, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 142, 143, 144, 145, 146, 147, 148, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 187, 188, 189, 190, 191, 192, 192, 193, 193, 194, 195,

(array([1]),)

After running the language model on the stimuli, reading and saving the hidden states as embeddings:

In [ ]:
for layer_id in np.arange(2):
    embeddings = {1:{},2:{},3:{},4:{}}
    for block_id in np.arange(1,5):
        
        print("phrases: ",len(translated_block_texts_steps[block_id]),translated_block_texts_steps[block_id][-1])
        print("scans: ",len(block_scans[block_id]), block_scans[block_id][-1].step)
        count = 0
        for scan in block_scans[block_id]:            
            step = scan.step
            real_step = step
            while real_step not in translated_block_texts_steps[block_id] and (real_step - step) < 3:
                real_step += 1
                print(real_step)
                
            if real_step in translated_block_texts_steps[block_id]:
                count += 1              
                embeddings[block_id][step] = []
                i = 0
                while i < 4 and (real_step+i) in translated_block_texts_steps[block_id]:
                    translated_steps = np.where(np.asarray(translated_block_texts_steps[block_id]) == real_step+i)[0]
                    print("translated steps %s, step %d, word %s" %(translated_steps,step,scan.all_words))
                    one_step_embeddings = []
                    for translated_step in translated_steps:
                        lstm_emb= np.load("../../lm_1b/new_output/subject_"+subject_id+"_block_"+str(block_id)+"_lstm_"+str(layer_id)+"_embeddings/lstm_emb_step_"+str(translated_step+1)+".npy")
                        print(lstm_emb)
                        one_step_embeddings.append(lstm_emb[0])
                    
                    embeddings[block_id][step].append(np.mean(one_step_embeddings,axis=0))
                    print(np.asarray(embeddings[block_id][step]).shape)
                    i += 1
                
        print(count)
        print(len(embeddings[block_id]))
        #print(embeddings[block_id].keys())
    np.save("../embeddings/subject_"+subject_id+"_lstm_"+str(layer_id)+"_emb_objects",embeddings)
    print(len(embeddings[1][0]))

In [ ]:
vocab_set = list(set(vocab))

In [ ]:
len(vocab_set)

In [ ]:
#read words:
with open("subject_"+subject_id+"_harry_potter_vocab.txt","r") as fin:
    print("Reading words...")
    words = fin.readlines()
    for i in np.arange(len(words)):
        words[i] = words[i].replace('\n',"")

In [ ]:
word_embeddings = np.load("../embeddings/subject_1/embeddings_char_cnn.npy")#("embeddings_char_cnn.npy")
len(word_embeddings)

In [ ]:
word_embedding_dic = {}
for i in np.arange(len(words)):
    word_embedding_dic[words[i]] = word_embeddings[i]
    
np.save("../embeddings/subject_"+subject_id+"_char_cnn_embedding_dic",word_embedding_dic)

In [ ]:
softmax_embeddings = np.load("../embeddings/subject_1/embeddings_softmax.npy")

In [ ]:
softmax_embeddings.shape

In [ ]:
all_vocab = []
softmax_vocab = []
with open("../../lm_1b/data/vocab-2016-09-10.txt","r") as vf:
    lines = vf.readlines()
    for line in lines:
        softmax_vocab.append(line.replace('\n',''))

In [ ]:
softmax_word_embedding_dic = {}
for i in np.arange(len(words)):
    if words[i] in softmax_vocab:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index(words[i])]
    elif words[i].lower() in softmax_vocab:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index(words[i].lower())]
    else:
        softmax_word_embedding_dic[words[i]] = softmax_embeddings[softmax_vocab.index('<UNK>')]
        
np.save("../embeddings/subject_"+subject_id+"_softmax_embedding_dic",word_embedding_dic)

In [ ]:
coords[:,0]

In [ ]:
brain3d = np.zeros((np.max(coords[:,0])+1,np.max(coords[:,1])+1,np.max(coords[:,2])+1))

In [ ]:
block_scans[1][2]

In [ ]:
block_id = 1
for k in np.arange(len(block_scans[block_id])):
    brain3d = np.zeros((np.max(coords[:,0])+1,np.max(coords[:,1])+1,np.max(coords[:,2])+1))    
    for i in np.arange(len(coords)):
        brain3d[coords[i][0],coords[i][1],coords[i][2]] = block_scans[block_id][k].activations[0][i]
    block_scans[block_id][k].brain3d = brain3d

In [ ]:
x,y,z = block_scans[1][0].brain3d.nonzero()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, -z, zdir='z', c= 'red')
plt.savefig("demo.png")


In [ ]:
np.save(processed_data_dir+"/subject_"+subject_id+"_extended_scan_object",block_scans)

In [ ]:
loaded_extended_scans = np.load(processed_data_dir+"/subject_"+subject_id+"_extended_scan_object.npy")

print(loaded_extended_scans.item().keys())
block_id = 1
print(len(loaded_extended_scans.item()[block_id]))
i = 0
print(loaded_extended_scans.item()[block_id][i].all_words)

In [ ]:
scan_objects = np.load("../processed_data/subject_1/subject_"+str(subject_id)+"_scan_objects.npy")
glove_embeddings = np.load("../embeddings/glove_word_embedding_dic.npy")
glove_embeddings.item()[""] = np.zeros_like(glove_embeddings.item()[list(glove_embeddings.item().keys())[0]])
glove_embeddings.item()["+"] = np.zeros_like(glove_embeddings.item()[list(glove_embeddings.item().keys())[0]])

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer


lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

block_ids = [1,2,3,4]

filtered_glove_embedding_dic = {}
filtered_glove_embedding_step_dic = {1:{},2:{},3:{},4:{}}
for block_id in block_ids:
    for scan_obj in scan_objects.item().get(block_id):
        words = []
        current_glove_words = []
        current_glove_embeddings = []
        for wordgroup in scan_obj.current_translated_words:
            words.extend(wordgroup)
            
                    
        if set(words).issubset(glove_embeddings.item().keys()):
            current_glove_words =  words
            for dic_word in current_glove_words:
                filtered_glove_embedding_dic[dic_word] = glove_embeddings.item()[dic_word]
        else:
            for w in words:
                if w in glove_embeddings.item().keys():
                    current_glove_words.append(w)
                    filtered_glove_embedding_dic[w] = glove_embeddings.item()[w]
                elif w.lower() in glove_embeddings.item().keys():
                    current_glove_words.append(w.lower())
                    filtered_glove_embedding_dic[w] = glove_embeddings.item()[w.lower()]
                else:
                    tokenized_w = nltk.word_tokenize(w.lower())
                    if set(tokenized_w).issubset(glove_embeddings.item().keys()):
                        current_glove_words.extend(tokenized_w)
                        for tok_w in tokenized_w:
                             filtered_glove_embedding_dic[w] = glove_embeddings.item()[tok_w]
                    else:
                        for t_w in tokenized_w:
                            if t_w in glove_embeddings.item().keys():
                                current_glove_words.append(t_w)
                                filtered_glove_embedding_dic[w] = glove_embeddings.item()[t_w]
                            else:
                                l_t_w = lemmatizer.lemmatize(t_w)
                                if l_t_w in glove_embeddings.item().keys():
                                    current_glove_words.append(l_t_w)
                                    filtered_glove_embedding_dic[w] = glove_embeddings.item()[l_t_w]
                                else:
                                    s_l_t_w = ps.stem(l_t_w)
                                    if s_l_t_w in glove_embeddings.item().keys():
                                        current_glove_words.append(s_l_t_w)
                                        filtered_glove_embedding_dic[w] = glove_embeddings.item()[s_l_t_w]
                                    else:
                                        print(w)
                                        filtered_glove_embedding_dic[w] = glove_embeddings.item()['unk']

        for w in current_glove_words:
            current_glove_embeddings.append(glove_embeddings.item()[w])
            
        filtered_glove_embedding_step_dic[block_id][scan_obj.step] = (current_glove_words,current_glove_embeddings)
            

In [ ]:

np.save("../embeddings/filtered_glove_embedding_step_dic",filtered_glove_embedding_step_dic)
np.save("../embeddings/filtered_glove_embedding_dic",filtered_glove_embedding_dic)

In [ ]:
glove_embeddings.item()['unk']